In [11]:

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from PIL import Image
from sklearn.decomposition import PCA
from tqdm import tqdm



# Functions and Constants

In [17]:

def get_file_list(directory) -> list:
    return os.listdir(directory)

def generate_dataframe(main_directory) -> pd.DataFrame:
    df = pd.DataFrame()
    labels_list = list()
    
    directory_list = get_file_list(main_directory)
    
    for directory in directory_list:
        folder_path = main_directory + "/" + directory
        file_list = get_file_list(folder_path)
        
        n_files = len(file_list)
        
        # Sample 10 files per type
        for i in tqdm(range(10)): #n_files
            file = file_list[i]
            file_path = folder_path + "/" + file
            
            img = Image.open(file_path)
            img_array = np.array(img)
            img_1d = np.reshape(img_array, -1)
            
            df_temp = pd.DataFrame(img_1d)
            df_temp = df_temp.T

            df = df.append(df_temp, ignore_index = True)
            labels_list.append(directory)
    
    df["label"] = pd.Series(labels_list)
    
    return df
    

# Load Data

In [22]:
df_oasis = None
df_pre = None

df_oasis = generate_dataframe("data/oasis")

100%|██████████| 10/10 [00:00<00:00, 155.04it/s]


In [21]:
df_oasis.loc[:, (df_oasis != 0).any(axis=0)]

,264,265,266,267,268,269,270,271,272,273,...,368655,368656,368657,368658,368659,368660,368661,368662,368663,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Mild_Dementia
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Mild_Dementia
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Mild_Dementia
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Mild_Dementia
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Mild_Dementia
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Mild_Dementia
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Mild_Dementia
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Mild_Dementia
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Mild_Dementia
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Mild_Dementia
